# Annual Metrics Sub-df Generation
This following generates a df of annual metric timeseries for each gage

In [6]:
# Imports -- Python 3.10
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import pyarrow as pa
import pyarrow.parquet as pq

from matplotlib import ticker
from importlib import reload
from dataretrieval import nwis
from datetime import timedelta
#import datetime

# Custom libs
import Src.func_ko as fn
import Src.classes as cl
reload(fn)
reload(cl)

<module 'Src.classes' from 'C:\\Users\\kondr\\Documents\\GitHub\\HighMagFlows_EPA_Project\\Src\\classes.py'>

In [7]:
date_range = 50
quantile = 95
test_limit = math.inf
DATASET_DTYPES = {'site_no': str, 'huc2_code': str, 'huc4_code': str, 'within_aq': str}

site_list_df = pd.read_excel(f'Prelim_Data/National_Metrics/National_Metrics_{date_range}_{quantile}.xlsx', dtype=DATASET_DTYPES)
site_list_df = site_list_df[site_list_df['valid'] == True] # Will run on valid sites only once national metrics is updated with tidal data
site_list = site_list_df['site_no'].tolist()
print(f'# of sites: {len(site_list)}')

# of sites: 3314


In [4]:
# list_results = []
# df_results = pd.DataFrame()

# #site_list = ['11447650', '13335050']
# for i, site in enumerate(site_list):
#     if i == test_limit: break
#     print(f'Working on site {site} ({i+1}/{len(site_list)})')
    
#     df = nwis.get_record(sites=site, service=fn.SERVICE, parameterCD=[fn.PARAM_CODE, fn.TIDAL_CODE], start=fn.DEFAULT_START, end=fn.DEFAULT_END)
#     df = df.reset_index()

#     # Only run on valid sites so this should never be the case but check anyways
#     #if df.empty: continue

#     if '00060_radar sensor_Mean' in df.columns and '00060_Mean' not in df.columns:
#         df.rename(columns={'00060_radar sensor_Mean': '00060_Mean'}, inplace=True)

#     df = fn.merge_tidal(df)

#     # Cropping to date range
#     date_threshold = pd.to_datetime(fn.DEFAULT_END).date() - timedelta(days=365.25 * date_range)
#     df = df[df['datetime'].dt.date >= date_threshold]

#     threshold = fn.calc_threshold(df, (quantile / 100))
#     hmf_series_defl, hmf_series_cont = fn.filter_hmf(df, threshold)
#     hmf_years = fn.num_hmf_years(hmf_series_defl) 

#     # Annual HMF 
#     df_hmf = hmf_series_cont.copy()
#     df_hmf = fn.convert_hmf(df_hmf, threshold)
#     df_hmf['00060_Mean'] = df_hmf['00060_Mean'].apply(lambda x: x * fn.CUBIC_FT_KM_FACTOR if x >= 0 else 0)
#     df_hmf.set_index('datetime', inplace=True)
#     df_hmf = df_hmf.resample(fn.HYDRO_YEAR).agg({'00060_Mean': ['sum']}).reset_index()
#     df_hmf.columns = ['water_year', 'annual_hmf']
#     df_hmf['water_year'] = df_hmf['water_year'].dt.year

#     # Duration and Event Metrics
#     hmf_series_cont = fn.convert_hmf(hmf_series_cont, threshold)
#     _, _, _, _, df_event_results = fn.calc_duration_intra_annual(hmf_series_cont, hmf_years)
#     df_event_results = df_event_results.drop(columns=['00060_Mean']).rename(columns={'Year': 'water_year', 'total_days': 'annual_duration', 'duration': 'event_duration'})
#     #print(df_event_results)

#     # Timing
#     hmf_series_defl = fn.convert_hmf(hmf_series_defl, threshold)
#     #print(hmf_series_defl)
#     _, com_series = fn.calc_timing(hmf_series_defl)
#     com_series = (pd.DataFrame(com_series).reset_index().rename(columns={'year': 'water_year', 'day': 'timing'}))
#     #print(com_series)

#     # Aggregation
#     df_temp = pd.merge(df_hmf, pd.merge(df_event_results, com_series, on='water_year', how='left'), on='water_year', how='left').reset_index(drop=True)#.fillna('NA') -- 'NA' not compatible with parquet
#     df_temp['site_no'] = site
#     df_temp = df_temp[['site_no', 'water_year', 'annual_hmf', 'annual_duration', 'event_hmf', 'event_duration', 'total_events', 'timing']]
#     #df_temp.to_csv('temp.csv')       

#     list_results.append(df_temp)
    
# df_results = pd.concat(list_results)
# #df_results.to_csv(f'annual_metrics_subdf_{date_range}_{quantile}.csv', index=False)
# pq.write_table(table=pa.Table.from_pandas(df_results), where=f'Prelim_Data/annual_metrics_subdf_{date_range}_{quantile}.parquet', compression='snappy')

In [8]:
list_results = []
df_results = pd.DataFrame()

#site_list = ['11447650', '13335050']
for i, site in enumerate(site_list):
    if i == test_limit: break
    print(f'Working on site {site} ({i+1}/{len(site_list)})')
    
    try:
        df = nwis.get_record(sites=site, service=fn.SERVICE, parameterCD=[fn.PARAM_CODE, fn.TIDAL_CODE], start=fn.DEFAULT_START, end=fn.DEFAULT_END)
        df = df.reset_index()
        
        # Only run on valid sites so this should never be the case but check anyways
        #if df.empty: continue
        
        if '00060_radar sensor_Mean' in df.columns and '00060_Mean' not in df.columns:
            df.rename(columns={'00060_radar sensor_Mean': '00060_Mean'}, inplace=True)
            
        df = fn.merge_tidal(df)
        
        # Cropping to date range
        date_threshold = pd.to_datetime(fn.DEFAULT_END).date() - timedelta(days=365.25 * date_range)
        df = df[df['datetime'].dt.date >= date_threshold]
        
        threshold = fn.calc_threshold(df, (quantile / 100))
        hmf_series_defl, hmf_series_cont = fn.filter_hmf(df, threshold)
        hmf_years = fn.num_hmf_years(hmf_series_defl) 
        
        # Annual HMF 
        df_hmf = hmf_series_cont.copy()
        df_hmf = fn.convert_hmf(df_hmf, threshold)
        df_hmf['00060_Mean'] = df_hmf['00060_Mean'].apply(lambda x: x * fn.CUBIC_FT_KM_FACTOR if x >= 0 else 0)
        df_hmf.set_index('datetime', inplace=True)
        df_hmf = df_hmf.resample(fn.HYDRO_YEAR).agg({'00060_Mean': ['sum']}).reset_index()
        df_hmf.columns = ['water_year', 'annual_hmf']
        df_hmf['water_year'] = df_hmf['water_year'].dt.year
        
        # Duration and Event Metrics
        hmf_series_cont = fn.convert_hmf(hmf_series_cont, threshold)
        _, _, _, _, df_event_results = fn.calc_duration_intra_annual(hmf_series_cont, hmf_years)
        df_event_results = df_event_results.drop(columns=['00060_Mean']).rename(columns={'Year': 'water_year', 'total_days': 'annual_duration', 'duration': 'event_duration'})
        #print(df_event_results)
        
        # Timing
        hmf_series_defl = fn.convert_hmf(hmf_series_defl, threshold)
        _, com_series = fn.calc_timing(hmf_series_defl)
        com_series = (pd.DataFrame(com_series).reset_index().rename(columns={'year': 'water_year', 'day': 'timing'}))
        #print(com_series)
        
        # Aggregation
        df_temp = pd.merge(df_hmf, pd.merge(df_event_results, com_series, on='water_year', how='left'), on='water_year', how='left').reset_index(drop=True)#.fillna('NA') -- 'NA' not compatible with parquet
        df_temp['site_no'] = site
        df_temp = df_temp[['site_no', 'water_year', 'annual_hmf', 'annual_duration', 'event_hmf', 'event_duration', 'total_events', 'timing']]
        #df_temp.to_csv('temp.csv')       
        
        list_results.append(df_temp)
        
    except Exception as e:
        print(f'ERROR: {e}')
        continue
    
df_results = pd.concat(list_results)
#df_results.to_csv(f'annual_metrics_subdf_{date_range}_{quantile}.csv', index=False)
pq.write_table(table=pa.Table.from_pandas(df_results), where=f'Prelim_Data/annual_metrics_subdf_{date_range}_{quantile}.parquet', compression='snappy')

Working on site 02342500 (1/3314)
Working on site 02361000 (2/3314)
Working on site 02369800 (3/3314)
Working on site 02371500 (4/3314)
Working on site 02372250 (5/3314)
Working on site 02373000 (6/3314)
Working on site 02374500 (7/3314)
Working on site 02376500 (8/3314)
Working on site 02399200 (9/3314)
Working on site 02401390 (10/3314)
Working on site 02412000 (11/3314)
Working on site 02414500 (12/3314)
Working on site 02419000 (13/3314)
Working on site 02421000 (14/3314)
Working on site 02422500 (15/3314)
Working on site 02424000 (16/3314)
Working on site 02425000 (17/3314)
Working on site 02428400 (18/3314)
Working on site 02448500 (19/3314)
Working on site 02450250 (20/3314)
Working on site 02456500 (21/3314)
Working on site 02464000 (22/3314)
Working on site 02465000 (23/3314)
Working on site 02467000 (24/3314)
Working on site 02467500 (25/3314)
Working on site 02469761 (26/3314)
Working on site 02469800 (27/3314)
Working on site 02471001 (28/3314)
Working on site 03574500 (29/

KeyboardInterrupt: 